In [1]:
from covid.simulator import Population
from covid.auxilliary import symptom_names
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from covid.policy import Policy

In [2]:
class BetaBernoulliModel:
    def __init__(self, nsymptom, nvacc):
        # Priors for the beta-bernoulli model
        self.a = np.ones(shape=[nvacc, nsymptom])/2 # using jeffreys prior
        self.b = np.ones(shape=[nvacc, nsymptom])/2 # using jeffreys prior
        self.nvacc = nvacc
        self.nsymptom = nsymptom

    def update(self, features, actions, outcomes):
        """
        for index in range(self.nvacc):
            print(outcomes[np.where(actions == (index-1))]) 
            print(actions)
            print(outcomes)
            if (np.sum(outcomes[np.where(actions == index - 1)], axis=1).size != 0):
                self.a[index] += np.sum(outcomes[np.where(actions == index - 1)], axis=1)
                self.b[index] += np.sum(outcomes[np.where(actions == index - 1)]==0, axis=1)\
                    - np.sum(outcomes[np.where(actions == index - 1)], axis=1)
            else: 
                self.b[index] += np.sum((outcomes==0)[np.where(actions == index - 1)], axis=1)
        """

        # performing the update to the beta - Bernoulli model for each case
        # in a loop. i.e. when |y| = 1
        # TODO: vecotirze this / fix the code above
        for index, outcome in enumerate(outcomes):
            self.a[int(actions[index])] += outcome
            self.b[int(actions[index])] += 1 - outcome

    def get_params(self):
        # Returns the parameters of all the beta distrobutions.
        return self.a, self.b


    def get_prob(self, features, action):
        # Returns the maximum posterior estiamte / mean of the theta-distrobution
        # which is a/(a+b) where a and b are the posterior parameters
        return self.a[action] / (self.a[action] + self.b[action])

    
    def retrain(self, features, actions, outcomes):
        # Retrains the model using jeffreys prior. The input should be the complete 
        # database of the trials etc. (This overwrites the old model)
        # With no privacy guarantee, this is equivivalent to continously update the model
        # However if we add laplace noise, then this would be a centralized method
        # as opposed to the local method which 'update' employs

        self.a = np.ones(shape=[self.nvacc, self.nsymptom])/2 # using jeffreys prior
        self.b = np.ones(shape=[self.nvacc, self.nsymptom])/2 # using jeffreys prior

        self.update(features, actions, outcomes)
    
    def set_default_action(self, action):
        self.default_action = action
    
    def set_default_symptom(self, symptom):
        self.default_symptom = symptom

    def predict(self, X):
        probs = self.get_prob(X, self.default_action)[:,self.default_symptom]
        return (probs > 0.5).astype(int)


In [3]:
class Naive(Policy):
    def get_utility(self, features, action, outcome):
        utility = 0
        for t, o in enumerate(outcome):
            utility -= np.dot(w, o)*(1+int(action[t] != -1))
       
        return utility

    def set_model(self, model):
        # the model needs to have a get_prob function which returns the probabilities of
        # each induvidual symptom

        self.model = model
    
    def get_action(self, features):
        """Get a completely random set of actions, but only one for each individual.

        If there is more than one individual, feature has dimensions t*x matrix, otherwise it is an x-size array.
        
        It assumes a finite set of actions.

        Returns:
        A t*|A| array of actions
        """
      
        
        n_obs = features.shape[0]
        actions = np.zeros(n_obs)
        
        for index, t in enumerate(features):
            u_list = []
            for a in self.action_set:
                u_list.append(self.get_expected_utility(a, t))
            actions[index] = np.argmax(np.array(u_list))
        return actions

    def get_expected_utility(self, action, features):
        p = self.model.get_prob(features, action)
        
        return -np.dot(p, w)*(1+int(action != -1))

    def observe(self, features, action, outcomes):
        self.model.update(features, action, outcomes)

        

In [4]:
## Baseline simulator parameters
n_genes = 128
n_vaccines = 3 # DO NOT CHANGE, breaks the simulator.
n_treatments = 1
n_population = 10000
n_symptoms = 10
batch_size = 1

w = np.array([0, 0.2, 0.1, 0.1, 0.1, 0.5, 0.2, 0.5, 1.0, 100])
#w = np.array([0, 1])

#assert n_population/batch_size == n_population//batch_size, 'the batch size must evenly divide the number of people'


# symptom names for easy reference
#from covid.auxilliary import symptom_names
population = Population(n_genes, n_vaccines, n_treatments)
#population.n_symptoms = 2

In [5]:
X = population.generate(n_population)
n_features = X.shape[1]

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [6]:
X.shape

(10000, 150)

In [7]:
# These are the possbile actions, currently restricted to -1, which is
# No vaccine, and 0, which is a vaccine
action_space = np.array([-1,1])
nactions = action_space.shape[0]

# Initializing our policy with the number of actions and
#vaccine_policy = NaiveWithTrial(nactions, action_space)
vaccine_policy = Naive(nactions, action_space)
#vaccine_policy.set_trial(1999)

# Here we set our predictive model
#vaccine_policy.set_model(LogisticModel(n_symptoms, nactions))
mod = BetaBernoulliModel(n_symptoms, nactions)
vaccine_policy.set_model(mod)

# Arrays to store the actions and outcomes for each
Y = np.zeros((n_population, n_symptoms))
A = np.zeros(n_population)


if batch_size == 1:
    for t in range(n_population):
        #print("Person nr: ", t)
        a_t = vaccine_policy.get_action(X[t].reshape((1, n_features)))
        # Then you can obtain results for everybody
        y_t = population.vaccinate([t], a_t.reshape((1, 1)))
        # Feed the results back in your policy. This allows you to fit the
        # statistical model you have.
        print('--------')
        print(X[t].shape)
        print(a_t.shape)
        print(y_t.shape)
        print('--------')
        vaccine_policy.observe(X[t], a_t, y_t)

        # Saving action taken and outcome obtained.
        Y[t] = y_t
        A[t] = a_t
else:
    for b in range(n_population//batch_size):
        print("Batch nr: ", b)
        start = b*batch_size
        stop = (b+1) * batch_size
        indexes = np.arange(start, stop, step=1).astype(int)

        a = vaccine_policy.get_action(X[indexes]).astype(int)
        #print(a)
        # Then you can obtain results for everybody
        y = population.vaccinate(indexes, a.reshape(batch_size, 1))

        # Feed the results back in your policy. This allows you to fit the
        # statistical model you have.
        vaccine_policy.observe(X[indexes], a, y)

        # Saving action taken and outcome obtained.
        Y[indexes] = y
        A[indexes] = a



Initialising policy with  2 actions
A = { [-1  1] }
TOTAL VACCINATED: 1
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 2
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 3
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 4
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 5
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 6
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 7
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 8
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 9
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 10
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 11
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 12
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 13
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 14
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 15
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINATED: 16
--------
(150,)
(1,)
(1, 10)
--------
TOTAL VACCINA

In [8]:
A

array([0., 0., 0., ..., 0., 0., 0.])

In [9]:
Y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [10]:
print(mod.a)
print(mod.b)

[[1.00005e+04 5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01 1.00005e+04
  5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01]
 [5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01
  5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01]]
[[5.00000e-01 1.00005e+04 1.00005e+04 1.00005e+04 1.00005e+04 5.00000e-01
  1.00005e+04 1.00005e+04 1.00005e+04 1.00005e+04]
 [5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01
  5.00000e-01 5.00000e-01 5.00000e-01 5.00000e-01]]


In [11]:
mod.set_default_action(A.astype(int))
mod.set_default_symptom(0)
mod.predict(X)

array([1, 1, 1, ..., 1, 1, 1])

In [12]:
from sklego.metrics import equal_opportunity_score

In [15]:
def equal_op(policy, X, A, Y, Z):
    if len(Y.shape) == 1:
        y_range = 1
    else:
        y_range = Y.shape[1]
    scores = np.zeros((len(Z), y_range))
    for i in range(len(Z)):
        for j in range(y_range):
            z = Z[i]
            #y = Y[j]
            if y_range == 1:
                y = 0
            else:
                y = j
            model = policy.model
            model.set_default_action(A.astype(int)) 
            model.set_default_symptom(y)
            #print(z, y)
            if y_range == 1:
                scores[i,j] = equal_opportunity_score(z)(model, X, Y)
            else:
                scores[i,j] = equal_opportunity_score(z)(model, X, Y[:,y])
    return scores

In [16]:
print(equal_op(vaccine_policy, X, A, Y, [3, 9]))

[[1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
